In [226]:
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

offset_ = 0
limit_ = 5000
url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/kn/object?offset={offset_}&limit={limit_}&sortField=devId.devShortCleanNm&sortType=asc&objStatus=0'
res = requests.get(url)
objects_data=res.json()
objects_list = objects_data.get('data').get('list')
objids = [x.get('objId') for x in objects_list] 

arr_obj = []
for i in range(0, len(objids)):
   url = f'https://xn--80az8a.xn--d1aqf.xn--p1ai/%D1%81%D0%B5%D1%80%D0%B2%D0%B8%D1%81%D1%8B/api/object/{objids[i]}'
   res = requests.get(url).json().get('data')
   dev = res.get('developer')
   del res['developer']
   
   key = 'photoRenderDTO'
   if key in res:
      del res[key]
   key = 'quartography'
   if key in res:
      del res[key]
   key = 'orgForm'
   if key in dev:
      del dev[key]
   
   res.update(dev)
   arr_obj.append(res)

with open("data_file.json", "w") as write_file:
   json.dump(arr_obj, write_file, sort_keys=False, indent=4, ensure_ascii=True, separators=(',', ': '))

base = pd.read_json('data_file.json', encoding='ISO-8859–1')
df = pd.DataFrame(base).fillna(0)

#экспорт в excel
excel_file = pd.ExcelWriter('save_pandas.xlsx')
df.to_excel(excel_file)
excel_file.save()

#экспорт в pickle
df.to_pickle('save_pandas.pkl')

#экспорт в БД
#engine = create_engine('sqlite:///save_pandas.db', echo=True)
#sqlite_connection = engine.connect()
#sqlite_table = "ЖК"
#df.to_sql(sqlite_table, sqlite_connection, if_exists='append')